**Import thư viện**


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import StandardScaler


**Đọc, chuẩn  bị dữ liệu, tensor**

In [ ]:

train_df = pd.read_csv("train.csv").dropna()
test_df  = pd.read_csv("test.csv").dropna()

X_train = train_df[["area", "bedrooms"]].values
y_train = train_df["price"].values.reshape(-1, 1)

X_test = test_df[["area", "bedrooms"]].values
y_test = test_df["price"].values.reshape(-1, 1)


scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)

X_test = scaler_X.transform(X_test)
y_test = scaler_y.transform(y_test)

# Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

Đặc trưng chọn: Area - Bedrooms

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)  # 2 đặc trưng

    def forward(self, x):
        return self.linear(x)

model = LinearRegressionModel()

criterion = nn.L1Loss()          # MAE loss
optimizer = optim.SGD(model.parameters(), lr=0.01)


epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    outputs = model(X_train)
    loss = criterion(outputs, y_train)

    loss.backward()
    optimizer.step()


model.eval()
with torch.no_grad():
    preds = model(X_test)
    mae = torch.mean(torch.abs(preds - y_test)).item()

    rmse = torch.sqrt(torch.mean((preds - y_test) ** 2)).item()

    ss_res = torch.sum((y_test - preds) ** 2)
    ss_tot = torch.sum((y_test - torch.mean(y_test)) ** 2)
    r2 = 1 - ss_res / ss_tot
    r2 = r2.item()

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)


w = model.linear.weight.data.numpy()
b = model.linear.bias.data.numpy()

print("Trọng số w:", w)
print("Bias b:", b)
